In [1]:
import cv2
import numpy as np

In [2]:
def detect_circles(frame):
    # Convert the frame to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the lower and upper boundaries of the orange color in HSV
    lower = np.array([160, 100, 100])
    upper = np.array([179, 255, 255])

    # Threshold the HSV image to get only orange colors
    mask = cv2.inRange(hsv, lower, upper)

    # Operations to remove noise and enhance the circles
    mask = cv2.erode(mask, None, iterations=5)
    mask = cv2.dilate(mask, None, iterations=5)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    circles = []

    # Loop over the contours and approximate the circles
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 100:  # Adjust the minimum area threshold as needed
            perimeter = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)

            # If the contour has a circular shape, add it to the list of circles
            if len(approx) >= 8:
                (x, y), radius = cv2.minEnclosingCircle(contour)
                circles.append((int(x), int(y), int(radius)))

                circularity = 4 * np.pi * area / (perimeter ** 2)
                if circularity > 0.95:
                    circles.append((int(x), int(y), int(radius)))

    return circles


In [4]:
# Open the video file
video_path = 'input.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Check if the video file was successfully opened
if not cap.isOpened():
    print('Error opening video file')
    exit()

# Read the first frame to get the video dimensions
_, frame = cap.read()
height, width, _ = frame.shape

# Create a VideoWriter object to save the output video
output_path = 'output.mp4'  # Replace with the desired output video file path
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output = cv2.VideoWriter(output_path, fourcc, 30.0, (width, height))

# Process each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect orange circles in the frame
    circles = detect_circles(frame)

    # Draw the detected circles on the frame
    # Draw rectangles around the detected circles
    for x, y, radius in circles:
        x = int(x)
        y = int(y)
        radius = int(radius)
        x1 = x - radius
        y1 = y - radius
        x2 = x + radius
        y2 = y + radius
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, "Circle", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Video', frame)


    # Write the frame to the output video file
    output.write(frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the resources
cap.release()
output.release()
cv2.destroyAllWindows()